<a href="https://colab.research.google.com/github/pedrogzz18/ASL-Model/blob/nn/NN_ASL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import string
letters = list(string.ascii_uppercase)
letters

['A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z']

In [12]:
numbers = [str(num) for num in range(0, 10)]
numbers

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

In [13]:
class_names = letters + numbers
class_names

['A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9']

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
import pandas as pd
from sklearn.metrics import accuracy_score

In [2]:
df = pd.read_csv('coords.csv')
df.head()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z19,v19,x20,y20,z20,v20,x21,y21,z21,v21
0,A,0.596015,0.598637,-5.351079e-07,0.0,0.524824,0.590247,0.003737,0.0,0.483084,...,0.008444,0.0,0.570060,0.442771,0.020445,0.0,0.584027,0.460907,0.033485,0.0
1,A,0.582702,0.602976,-4.789155e-07,0.0,0.514781,0.599014,0.001139,0.0,0.469273,...,0.004532,0.0,0.555620,0.450476,0.013084,0.0,0.570349,0.470588,0.025043,0.0
2,A,0.568719,0.601903,-4.693277e-07,0.0,0.502747,0.596997,0.000186,0.0,0.459650,...,-0.004795,0.0,0.544712,0.446647,0.002810,0.0,0.558533,0.469868,0.014419,0.0
3,A,0.559268,0.600854,-4.881787e-07,0.0,0.491037,0.593645,0.006739,0.0,0.447823,...,0.004258,0.0,0.532012,0.447075,0.011150,0.0,0.547573,0.469695,0.022285,0.0
4,A,0.548125,0.613788,-4.889142e-07,0.0,0.476834,0.605301,0.008348,0.0,0.434377,...,0.007032,0.0,0.514065,0.463542,0.014416,0.0,0.530254,0.481641,0.025965,0.0


In [3]:
df.dropna(axis = 0, inplace = True)

In [4]:
X = df.drop('class', axis = 1)
y = df['class']

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42, stratify = y)

In [6]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import InputLayer, Dense

In [7]:
def create_asl_nn():
  model = Sequential([
      Dense(input_shape=(84,), units=84, activation='relu'),
      Dense(units=32, activation='relu'),
      Dense(units=16, activation='relu'),
      Dense(units=len(class_names), activation='softmax')
  ])
  model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=["accuracy"])
  return model

In [32]:
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score
from tensorflow.keras.callbacks import EarlyStopping

trans = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
clf = KerasClassifier(create_asl_nn, epochs=100, batch_size=200, shuffle=True, verbose=1, validation_split=0.2 ,callbacks=[EarlyStopping(min_delta=0.001, patience=5)])
clf = make_pipeline(StandardScaler(), clf)

In [33]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y_train_n = label_encoder.fit_transform(y_train)
clf.fit(X_train, y_train_n)

Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


41/41 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.1050 - loss: 3.4341 - val_accuracy: 0.2773 - val_loss: 2.7735
Epoch 2/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3401 - loss: 2.4661 - val_accuracy: 0.6292 - val_loss: 1.4795
Epoch 3/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6920 - loss: 1.2431 - val_accuracy: 0.8196 - val_loss: 0.7415
Epoch 4/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8494 - loss: 0.6514 - val_accuracy: 0.9066 - val_loss: 0.4452
Epoch 5/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9175 - loss: 0.4033 - val_accuracy: 0.9418 - val_loss: 0.3008
Epoch 6/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9400 - loss: 0.2783 - val_accuracy: 0.9493 - val_loss: 0.2159
Epoch 7/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9538 - loss: 0.2002 - val_accuracy: 0.9642 - val_loss: 0.1642
Epoch 8/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9672 - loss: 0.1555 - val_accuracy: 0.9692 - val_loss: 0.1

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('kerasclassifier',
                 KerasClassifier(batch_size=200, callbacks=[<keras.src.callbacks.early_stopping.EarlyStopping object at 0x7850864e63b0>], epochs=100, model=<function create_asl_nn at 0x78508e5b2440>, validation_split=0.2))])

In [23]:
preds = clf.predict(X_test)
preds = label_encoder.inverse_transform(y=preds)
acc = accuracy_score(y_test, preds)
print('NN', acc)

22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
NN 0.9935034802784223


In [26]:
label_encoder.inverse_transform(clf.predict(pd.DataFrame([X_test.iloc[0][:]])))[0]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step


'I'

In [27]:
max(clf.predict_proba(pd.DataFrame([X_test.iloc[0][:]])))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step


0.9972435